In [ ]:
from lec_utils import *
def show_grouping_animation():
    src = "https://docs.google.com/presentation/d/1tBaFyHseIGsX5wmE3BdNLeVHnKksQtpzLhHge8Tzly0/embed?start=false&loop=false&delayms=60000&rm=minimal"
    width = 960
    height = 509
    display(IFrame(src, width, height))

<div class="alert alert-info" markdown="1">

#### Lecture 5

# Querying and Grouping

### EECS 398-003: Practical Data Science, Fall 2024

<small><a style="text-decoration: none" href="https://practicaldsc.org">practicaldsc.org</a> • <a style="text-decoration: none" href="https://github.com/practicaldsc/fa24">github.com/practicaldsc/fa24</a></small>
    
</div>

### Announcements 📣

- Homework 2 is due on **Thursday**.
<br><small>Post on [Ed](https://edstem.org/us/courses/61012/discussion/) or
come to [Office Hours](https://practicaldsc.org/calendar) for help! We're using a queue for office hours now – access it from [practicaldsc.org/calendar](https://practicaldsc.org/calendar).</small>

- [**study.practicaldsc.org**](https://study.practicaldsc.org) contains our discussion worksheets (and solutions), which are made up of old exam problems. Use these problems to build your theoretical understanding of the material!

- Homework 1 scores are available on Gradescope.

### Agenda

- Recap: Querying.
- Adding and modifying columns.
- `pandas` and `numpy`.
- Introduction to the `groupby` method.
- `groupby`'s inner workings.

Remember to follow along in lecture by accessing the "blank" lecture notebook in our [public GitHub repository](https://github.com/practicaldsc/fa24).

<div class="alert alert-warning">
    <h3>Question 🤔 (Answer at <a style="text-decoration: none; color: #0066cc" href="https://docs.google.com/forms/d/e/1FAIpQLSd4oliiZYeNh76jWy-arfEtoAkCrVSsobZxPwxifWggo3EO0Q/viewform">practicaldsc.org/q</a>)</h3>
    
<small>Remember that you can always ask questions anonymously at the link above!</small>

## Querying

How do we find rows that satisfy certain **conditions**?

---

Run the cell below to load in our dataset.

In [ ]:
dogs = pd.read_csv('data/dogs43.csv').set_index('breed')
dogs.head()

### Recap: Querying

- Querying is the act selecting rows in a DataFrame that satisfy certain condition(s).<br><small>We sometimes call this "filtering."</small>

- Example: How many breeds live to be over 10 years old?

In [ ]:
dogs[dogs['longevity'] > 10].shape[0] 

- Example: Among all breeds with `'Retriever'` in the same, which is the second tallest?<br><small>Remember, we like to format our code this way when writing long, hard-to-read lines.</small>

In [ ]:
(
    dogs.loc[dogs.index.str.contains('Retriever'), 'height']
    .sort_values(ascending=False)
    .iloc[1]
)

- What is the distribution of breeds `'size'`s among `'sporting'` and `'working'` breeds?<br><small>Remember, you need parentheses around each condition. Also, you must use the bitwise operators `&` and `|` instead of the standard `and` and `or` keywords, as we saw in Lecture 3.</small>

In [ ]:
(
    dogs.loc[(dogs['kind'] == 'sporting') | (dogs['kind'] == 'working'), 'size']
    .value_counts()
)

In [ ]:
# Equivalent to the above!
(
    dogs.loc[dogs['kind'].isin(['sporting', 'working']), 'size']
    .value_counts()
)

- Show me all rows for `'medium'`-sized dogs.

In [ ]:
dogs[dogs['size'] == 'medium'] 

- Show me all rows for `'Golden Retriever'`s.<br><small>Note that because we set the index to `'breed'` earlier, we can select rows based on dog breeds without having to query. If `'breed'` was instead a column, then we'd need to query to access information about a particular breed.</small>

In [ ]:
dogs.loc['Golden Retriever'] 

<div class="alert alert-danger">
    <h3>Aside: Reference Slides</h3>

- Moving forward, I'm going to try and spend a bit less time on syntax and a bit more time on conceptual problem-solving.

- So, in each lecture, some slides will be called "Reference Slides".

- **We will skip these slides during live lecture, but they'll be present in the posted lecture notebooks, so that you can look at them when working on activities in class and on discussion and homework problems**.<br><small>The material in them is in-scope.</small>

- Reference slides will appear with red headers, like this one.<br><small>There's a Reference Slide right after this slide, about the DataFrame `query` method!</small>

<div class="alert alert-danger" markdown="1">

#### Reference Slide

### The `query` method

The DataFrame `query` method is a convenient way to query, since you don't need parentheses and you can use the `and` and `or` keywords.

In [ ]:
dogs

In [ ]:
dogs.query('weight < 20 and kind == "terrier"')

In [ ]:
dogs.query('kind in ["sporting", "terrier"] and lifetime_cost < 20000')

<div class="alert alert-danger" markdown="1">

#### Reference Slide

### More practice

In [ ]:
jack = pd.DataFrame({1: ['fee', 'fi'], 
                     '1': ['fo', 'fum']})
jack

For each of the following pieces of code, predict what the output will be. Then, uncomment the line of code and see for yourself. We **won't** cover these in class, but you should try them out yourself. [Here's a Pandas Tutor link](https://pandastutor.com/vis.html#code=import%20pandas%20as%20pd%0A%0Ajack%20%3D%20pd.DataFrame%28%7B1%3A%20%5B'fee',%20'fi'%5D,%20%0A%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20'1'%3A%20%5B'fo',%20'fum'%5D%7D%29%0Ajack%5B1%5D&d=2023-10-05&lang=py&v=v1) to visualize these!

In [ ]:
# jack[1]

In [ ]:
# jack[[1]]

In [ ]:
# jack['1']

In [ ]:
# jack[[1, 1]]

In [ ]:
# jack.loc[1]

In [ ]:
# jack.loc[jack[1] == 'fo']

In [ ]:
# jack[1, ['1', 1]]

In [ ]:
# jack.loc[1,1]

## Adding and modifying columns

---

### Adding and modifying columns, using a copy

- To add a new column to a DataFrame, use the `assign` method.<br><small>To change the values in a column, add a new column with the same name as the existing column.</small>

- Like most `pandas` methods, `assign` returns a new DataFrame.<br><small>
    - **Pro** ✅: This doesn't inadvertently change any existing variables.
    - **Con** ❌: It is not very space efficient, as it creates a new copy each time it is called.</small>

In [ ]:
dogs.assign(cost_per_year=dogs['lifetime_cost'] / dogs['longevity']) 

In [ ]:
dogs

<div class="alert alert-danger" markdown="1">

#### Reference Slide

### `assign` for column names with special characters

You can also use `assign` when the desired column name has spaces (and other special characters) by unpacking a dictionary:

In [ ]:
dogs.assign(**{'cost per year 💵': dogs['lifetime_cost'] / dogs['longevity']})

### Adding and modifying columns, in-place

- You can assign a new column to a DataFrame **in-place** using `[]`.<br><small>This works like dictionary assignment. Using `[]` **modifies** the underlying DataFrame, unlike `assign`, which returns a new DataFrame.</small>

- This is the more "common" way of adding/modifying columns.<br><small>**Exercise caution when using this approach, since it is destructive – it changes the values of existing variables.**</small>

In [ ]:
# By default, .copy() returns a deep copy of the object it is called on,
# meaning that if you change the copy, the original remains unmodified.
dogs_copy = dogs.copy() 
dogs_copy.head(2)  NO PROMPT

In [ ]:
dogs_copy['cost_per_year'] = dogs_copy['lifetime_cost'] / dogs_copy['longevity']
dogs_copy

- Note that we never reassigned `dogs_copy` in the cell above – that is, we never wrote `dogs_copy = ...` – though it was still modified.

### Mutability

- DataFrames, like lists, arrays, and dictionaries, are **mutable**. As we saw in Lecture 2, this means they can be modified in-place after creation.

- Not only does this explain the behavior on the previous slide, but it also explains the following:

In [ ]:
dogs_copy

In [ ]:
def cost_in_thousands():
    dogs_copy['lifetime_cost'] = dogs_copy['lifetime_cost'] / 1000

In [ ]:
# What happens when we run this twice? Three times?
cost_in_thousands()

In [ ]:
dogs_copy

### ⚠️ Warning: Avoid mutation when possible!

- Note that `dogs_copy` was modified, even though we didn't reassign it! These unintended consequences can **influence the behavior of test cases on homeworks**, among other things!

- To avoid this, it's a good idea to avoid mutation when possible. If you must use mutation, include `df = df.copy()` as the first line in functions that take DataFrames as input.

<div class="alert alert-warning">
    <h3>Question 🤔 (Answer at <a style="text-decoration: none; color: #0066cc" href="https://docs.google.com/forms/d/e/1FAIpQLSd4oliiZYeNh76jWy-arfEtoAkCrVSsobZxPwxifWggo3EO0Q/viewform">practicaldsc.org/q</a>)</h3>
    
<small>Remember that you can always ask questions anonymously at the link above!</small>

## `pandas` and `numpy`

---

<center><img src='imgs/python-stack.png' width=60%></center>

### `pandas` is built upon `numpy`!

- A Series in `pandas` is a `numpy` array with an index.

- A DataFrame is like a dictionary of columns, each of which is a `numpy` array.

- Many operations in `pandas` are fast because they use `numpy`'s implementations, which are written in fast, compiled languages like C.

- If you need to access the array underlying a DataFrame or Series, use the `to_numpy` method.

In [ ]:
dogs['lifetime_cost']

In [ ]:
dogs['lifetime_cost'].to_numpy() 

<div class="alert alert-danger" markdown="1">

#### Reference Slide

### `pandas` data types

- Each Series (column) has a `numpy` data type, which refers to the type of the values stored within. Access it using the `dtypes` attribute.

- A column's data type determines which operations can be applied to it.

- `pandas` tries to guess the correct data types for a given DataFrame, and is often wrong.
    - This can lead to incorrect calculations and poor memory/time performance.

- As a result, you will often need to explicitly convert between data types.

In [ ]:
dogs

In [ ]:
dogs.dtypes

<div class="alert alert-danger" markdown="1">

#### Reference Slide

### `pandas` data types

- Notice that Python `str` types are `object` types in `numpy` and `pandas`.

|Pandas dtype|Python type|NumPy type|SQL type|Usage|
|---|---|---|---|---|
|int64|int|int_, int8,...,int64, uint8,...,uint64|INT, BIGINT| Integer numbers|
|float64|float|float_, float16, float32, float64|FLOAT| Floating point numbers|
|bool|bool|bool_|BOOL|True/False values|
|datetime64 or Timestamp|datetime.datetime|datetime64|DATETIME|Date and time values|
|timedelta64 or Timedelta|datetime.timedelta|timedelta64|NA|Differences between two datetimes|
|category|NA|NA|ENUM|Finite list of text values|
|object|str|string, unicode|NA|Text|
|object|NA|object|NA|Mixed types|

- [This article](https://www.dataquest.io/blog/pandas-big-data/) details how `pandas` stores different data types under the hood.

- [This article](https://mortada.net/can-integer-operations-overflow-in-python.html#Can-integers-overflow-in-python?) explains how `numpy`/`pandas` `int64` operations differ from vanilla `int` operations.

<div class="alert alert-danger" markdown="1">

#### Reference Slide

### Type conversion

- You can change the data type of a Series using the `.astype` Series method.

- For example, we can change the data type of the `'lifetime_cost'` column in `dogs` to be `uint32`:

In [ ]:
dogs

In [ ]:
# Gives the types as well as the space taken up by the DataFrame.
dogs.info()

In [ ]:
dogs['lifetime_cost'] = dogs['lifetime_cost'].astype('uint32')

- Now, the DataFrame takes up less space! This may be insignificant in our DataFrame, but makes a difference when working with larger datasets.

In [ ]:
dogs.info()

<div class="alert alert-danger" markdown="1">

#### Reference Slide

### Setting `dtype`s in `read_csv`

Usually, we prefer to set the correct dtypes in `read_csv`, since it can help `pandas` load in files more quickly:

In [ ]:
dogs_new = pd.read_csv('data/dogs43.csv', dtype={'lifetime_cost': 'uint32'})
dogs_new

In [ ]:
dogs_new.dtypes

### Axes

- The rows and columns of a DataFrame are both stored as Series.

- The **axis** specifies the direction of a **slice** of a DataFrame.

<center><img src='imgs/axis.png' width=30%></center>

- Axis 0 refers to the rows and axis 1 refers to the columns.<br><small>**These are the same axes definitions that 2D `numpy` arrays have!**</small>

### DataFrame methods with `axis`

- Many Series methods work on DataFrames.

- In such cases, the DataFrame method usually applies the Series method to every row or column.

- Many of these methods accept an `axis` argument; the default is usually `axis=0`.

In [ ]:
dogs

In [ ]:
# Max element in each column.
dogs.max() 

In [ ]:
# Max element in each row – a little nonsensical, since the values in each column are on different scales.
# Note that we had to select the numeric columns first.
dogs[['lifetime_cost', 'longevity', 'weight', 'height']].max(axis=1)

In [ ]:
# The number of unique values in each column.
dogs.nunique() 

In [ ]:
# describe doesn't accept an axis argument; it works on every numeric column in the DataFrame it is called on.
dogs.describe() 

<div class="alert alert-success">
<h3>Activity</h3>

Pick a dog breed that you personally like or know the name of. Then:
<ul>
    <li>Try to find a few other dog breeds that are similar in weight to yours in <code>all_dogs</code>.</li>
    <li>Which similar breeds have the lowest and highest <code>'lifetime_cost'</code>? <code>'intelligence_rank'</code>?</li>
    <li>Are there any similar breeds that you haven't heard of before?</li>
</ul>
<br>
    For fun, look up these dog breeds on the <a href="https://www.akc.org/">AKC website</a> to see what they look like!
</div>

In [ ]:
all_dogs = pd.read_csv('data/all_dogs.csv')
all_dogs

In [ ]:
# There's no "right answer" here; you're supposed to explore!
fav_weight = all_dogs.loc[all_dogs['breed'] == 'English Cocker Spaniel', 'weight'].iloc[0]
similar_weight = all_dogs[(all_dogs['weight'] >= fav_weight - 5) & (all_dogs['weight'] <= fav_weight + 5)]
similar_weight

In [ ]:
similar_weight.sort_values('intelligence_rank')[['breed', 'lifetime_cost', 'intelligence_rank']] 

## Introduction to the `groupby` method

---

### Example: Palmer Penguins

<center><img src="imgs/lter_penguins.png" width=60%>
<i><a href="https://github.com/allisonhorst/palmerpenguins/blob/main/README.md">Artwork by @allison_horst</a></i>

</center>

The dataset we'll work with for the rest of the lecture involves various measurements taken of three species of penguins in Antarctica.

In [ ]:
IFrame('https://www.youtube-nocookie.com/embed/CCrNAHXUstU?si=-DntSyUNp5Kwitjm&amp;start=11',
       width=560, height=315)

### Loading the data

In [ ]:
penguins = sns.load_dataset('penguins').dropna().reset_index(drop=True)
penguins

- Here, each row corresponds to a single penguin, and each column corresponds to a different attribute (or feature) we have for each penguin.

- Data formatted in this way is sometimes called [tidy data](https://r4ds.had.co.nz/tidy-data.html).

### Visualizing the data

In [ ]:
penguins.plot(kind='scatter', 
              x='bill_length_mm', 
              y='body_mass_g', 
              color='species', 
              title='Body Mass vs. Bill Length')

### Granularity

- Granularity refers to what each observation in a dataset represents.
    - Fine: small details.
    - Coarse: bigger picture.

- If you can control how your dataset is created, you should opt for **finer granularity**, i.e. for more detail.
    - You can always remove details, but it's difficult to add detail that isn't already there.
    - But obtaining fine-grained data can take more time/money.

- Today, we'll focus on how to **remove** details from fine-grained data, in order to help us understand bigger-picture trends in our data.

### Aggregating

- **Aggregating** is the act of combining many values into a single value.

- What is the mean `'body_mass_g'` for all penguins?

In [ ]:
penguins['body_mass_g'].mean() 

- What is the mean `'body_mass_g'` **for each `'species'`**?

### A naïve approach to finding the mean `'body_mass_g'` per `'species'`

- First, we could identify all unique values in the `'species'` column.

In [ ]:
penguins['species'].unique() 

- Then, for each `'species'`, we could:
    1. Query for just that `'species'`.
    1. Extract the `'body_mass_g'` column and use the `mean` method on it.

In [ ]:
penguins.loc[penguins['species'] == 'Adelie', 'body_mass_g'].mean() 

In [ ]:
penguins.loc[penguins['species'] == 'Chinstrap', 'body_mass_g'].mean() 

In [ ]:
penguins.loc[penguins['species'] == 'Gentoo', 'body_mass_g'].mean() 

- We _could_ use a `for`-loop, but remember, we want to avoid Python `for`-loops.

### Grouping

- A better solution is to use the `groupby` method.

In [ ]:
# To find the overall mean 'body_mass_g':
penguins['body_mass_g'].mean() 

In [ ]:
# To find the mean 'body_mass_g' for each 'species':
penguins.groupby('species')['body_mass_g'].mean() 

- Somehow, the `groupby` method computes what we're looking for in just one line. How?

- We'll work through the internals, but remember this: **if you need to calculate something _for each group_, use `groupby`!**

### An illustrative example: Pets 🐱 🐶🐹

- Consider the DataFrame `pets`, shown below.

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>Species</th>
      <th>Color</th>
      <th>Weight</th>
      <th>Age</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>dog</td>
      <td>black</td>
      <td>40</td>
      <td>5.0</td>
    </tr>
    <tr>
      <th>1</th>
      <td>cat</td>
      <td>golden</td>
      <td>15</td>
      <td>8.0</td>
    </tr>
    <tr>
      <th>2</th>
      <td>cat</td>
      <td>black</td>
      <td>20</td>
      <td>9.0</td>
    </tr>
    <tr>
      <th>3</th>
      <td>dog</td>
      <td>white</td>
      <td>80</td>
      <td>2.0</td>
    </tr>
    <tr>
      <th>4</th>
      <td>dog</td>
      <td>golden</td>
      <td>25</td>
      <td>0.5</td>
    </tr>
    <tr>
      <th>5</th>
      <td>hamster</td>
      <td>golden</td>
      <td>1</td>
      <td>3.0</td>
    </tr>
  </tbody>
</table>

- Let's see what happens under the hood when we use the `groupby` method on `pets`.

In [ ]:
show_grouping_animation()

### Let's try it out!

In [ ]:
pets = pd.DataFrame().assign(
    Species=['dog', 'cat', 'cat', 'dog', 'dog', 'hamster'],
    Color=['black', 'golden', 'black', 'white', 'golden', 'golden'],
    Weight=[40, 15, 20, 80, 25, 1],
    Age=[5, 8, 9, 2, 0.5, 3]
)
pets

In [ ]:
# Why does this error?
pets.groupby('Species').mean()

In [ ]:
pets.groupby('Species')[['Weight', 'Age']].mean() 

In [ ]:
pets.groupby('Species').max() 

It takes several steps to go from the original `pets` DataFrame to this grouped DataFrame, but we don't get to see any of Python's inner workings, just the final output.

### "Split-apply-combine" paradigm

- The `groupby` method involves three steps: **split**, **apply**, and **combine**.<br><small>This is the same terminology that the [`pandas` documentation](https://pandas.pydata.org/docs/user_guide/groupby.html) uses.</small>

<center><img src="imgs/image_0.png" width=40%></center>

- **Split** breaks up and "groups" the rows of a DataFrame according to the specified **key**.<br><small>There is one "group" for every unique value of the key.</small>

- **Apply** uses a function (e.g. aggregation, transformation, filtration) within the individual groups.

- **Combine** stitches the results of these operations into an output DataFrame.

- The split-apply-combine pattern can be **parallelized** to work on multiple computers or threads, by sending computations for each group to different processors.

### More examples

Before we dive into the internals, let's look at a few more examples.

<div class="alert alert-success">
<h3>Activity</h3>
        
Which `'species'` has the highest median `'bill_length_mm'`?

In [ ]:
(
    penguins
    .groupby('species')
    ['bill_length_mm']
    .median()
    .idxmax()
)

In [ ]:
(
    penguins
    .groupby('species')
    ['bill_length_mm']
    .median()
    .plot(kind='barh', title='Median Bill Length of Each Species')
)

<div class="alert alert-success">
<h3>Activity</h3>
                
What proportion of penguins of each `'species'` live on `'Dream'` island?

</div>

In [ ]:
(
    penguins
    .assign(is_Dream=penguins['island'] == 'Dream')
    .groupby('species')
    ['is_Dream']
    .mean()
)

## `groupby`'s inner workings

---

### How does `groupby` actually work?

- We've just evaluated a few expressions of the following form.

In [ ]:
penguins.groupby('species')['bill_length_mm'].mean()

- There are three "building blocks"
in the above expression:
    1. `penguins.groupby('species')`.<br><small>First, we specify which column we want to group on.</small>
    1. `['bill_length_mm']`.<br><small>Then, we select the other relevant columns for our calculations.</small>
    1. `.mean()`.<br><small>Finally, we use an aggregation method.</small>

- Let's see what each block contributes to the output.

### `DataFrameGroupBy` objects

<pre>

<strong><span style="color: #0066cc">penguins.groupby('species')</span></strong><span style="color: #999">['bill_length_mm'].mean()</span>

</pre>

- If `df` is a DataFrame, then `df.groupby(key)` returns a `DataFrameGroupBy` object.<br><small>This object represents the "split" in "split-apply-combine".</small>

In [ ]:
# Simplified DataFrame for demonstration:
penguins_small = penguins.iloc[[0, 150, 300, 1, 251, 151, 301], [0, 5, 6]]
penguins_small

In [ ]:
# Creates one group for each unique value in the species column.
penguins_small.groupby('species')

- `DataFrameGroupBy` objects have a `groups` attribute, which is a dictionary in which the keys are group names and the values are lists of row labels.<br><small>We won't actually use this, but it's helpful in understanding how `groupby` works under-the-hood.</small>

In [ ]:
penguins_small.groupby('species').groups

### Column extraction

<pre>

<span style="color: #999">penguins.groupby('species')</span><strong><span style="color: #0066cc">['bill_length_mm']</span></strong><span style="color: #999">.mean()</span>

</pre>

- After creating a `DataFrameGroupBy` object, we typically select the relevant column(s) that we want to aggregate.

- The result is either a `SeriesGroupBy` or `DataFrameGroupBy` object, depending on what's passed in.

In [ ]:
penguins.groupby('species')['bill_length_mm'] 

In [ ]:
penguins.groupby('species')[['bill_length_mm', 'bill_depth_mm']] 

- As we've seen already, you should the columns you want to aggregate **before** using your aggregation method, or you may run into errors!

In [ ]:
# This errors, because there are non-numeric columns in penguins
# that it's trying to take the "mean" of, like 'island'.
penguins.groupby('species').mean() 

In [ ]:
penguins.groupby('species')[['bill_length_mm', 'bill_depth_mm']].mean() 

### Aggregation

<pre>

<span style="color: #999">penguins.groupby('species')['bill_length_mm']</span><strong><span style="color: #0066cc">.mean()</span></strong>

</pre>

- Once we create a `DataFrameGroupBy` or `SeriesGroupBy` object, we need to **apply** some function to each group, and **combine** the results.

- The most common operation we apply to each group is an **aggregation**, but we'll see examples of **filtrations** and **transformations** soon<br><small>Remember, aggregation is the act of combining many values into a single value.</small>

- To perform an aggregation, use an aggregation method on the `DataFrameGroupBy` or `SeriesGroupBy` object, e.g. `.mean()`, `.max()`, or `.median()`.

Let's look at some examples.

In [ ]:
penguins_small

In [ ]:
penguins_small.groupby('species')['body_mass_g'].mean() 

In [ ]:
# Note that this worked on the entire DataFrame!
# But, if all we wanted are the sums of `'body_mass_g'
# for each species, this is slower than
# penguins_small.groupby('species')['body_mass_g'].mean().
penguins_small.groupby('species').sum() 

In [ ]:
# Often used in conjunction with sort_values.
# Remember this when you work on the activity in a few slides!
penguins_small.groupby('species').last() 

In [ ]:
penguins_small.groupby('species').max() 

### Column independence

- Within each group, the aggregation method is applied to **each column independently**.

In [ ]:
penguins_small.groupby('species').max()

- The above result **is not** telling us that there is a `'Male'` `'Adelie'` penguin with a `'body_mass_g'` of `3800.0`!

In [ ]:
# This penguin is Female!
penguins_small.loc[(penguins['species'] == 'Adelie') & (penguins['body_mass_g'] == 3800.0)]

<div class="alert alert-success">
<h3>Activity</h3>

Find the <code>'species'</code>, <code>'island'</code>, and <code>'body_mass_g'</code> of the heaviest <code>'Male'</code> and <code>'Female'</code> penguins in <code>penguins</code> (not <code>penguins_small</code>).
</div>

In [ ]:
# General idea: Sort the penguibs by mass in decreasing order.
# Then, the first male penguin that appears is the heaviest male penguin,
# and the first female penguin that appears is the heaviest female penguin.
# For each sex, take the first row.
(
    penguins
    .sort_values('body_mass_g', ascending=False)
    .groupby('sex')
    .first()
)

### Beyond default aggregation methods

- There are many built-in aggregation methods.

- What if you want to apply different aggregation methods to different columns?

- What if the aggregation method you want to use doesn't already exist in `pandas`?

### The `aggregate` method

- `DataFrameGroupBy` and `SeriesGroupBy` objects have a general `aggregate` method, which aggregates using one or more operations.<br><small>Remember, aggregation is the act of combining many values into a single value.</small>

- There are many ways of using `aggregate`; refer to [the documentation](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.core.groupby.DataFrameGroupBy.aggregate.html) for a comprehensive list.<br><small>Per [the documentation](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.agg.html), `agg` is an alias for `aggregate`.</small>

- Example arguments:
    - A single function.
    - A list of functions.
    - A dictionary mapping column names to functions.

- We've attached a Reference Slide with examples.

<div class="alert alert-danger" markdown="1">

#### Reference Slide

### Examples

- How many penguins are there of each `'species'`, and what is the mean `'body_mass_g'` of each `'species'`?

In [ ]:
(
    penguins
    .groupby('species')
    ['body_mass_g']
    .aggregate(['count', 'mean'])
)

- What is the maximum `'bill_length_mm'` of each `'species'`, and which `'island'`s is each `'species'` found on?

In [ ]:
(
    penguins
    .groupby('species')
    .agg({'bill_length_mm': 'max', 'island': 'unique'})
)

<div class="alert alert-success">
<h3>Activity</h3>

What is the **interquartile range** of the `'body_mass_g'` of each `'species'`?
    
The interquartile range of a distribution is defined as:
    
$$\text{75th percentile} - \text{25th percentile}$$
    
***Hint***: Use `np.percentile`, and pass `agg`/`aggregate` a custom function.

In [ ]:
# Here, the argument to agg is a function,
# which takes in a Series and returns a scalar.
def iqr(s):
    return np.percentile(s, 75) - np.percentile(s, 25)
(
    penguins
    .groupby('species')
    ['body_mass_g']
    .agg(iqr)
)

<div class="alert alert-warning">
    <h3>Question 🤔 (Answer at <a style="text-decoration: none; color: #0066cc" href="https://docs.google.com/forms/d/e/1FAIpQLSd4oliiZYeNh76jWy-arfEtoAkCrVSsobZxPwxifWggo3EO0Q/viewform">practicaldsc.org/q</a>)</h3>
    
<small>Remember that you can always ask questions anonymously at the link above!</small>
    
What questions do you have?

### Split-apply-combine, revisited

- When we introduced the split-apply-combine pattern, the "apply" step involved **aggregation** – our final DataFrame had one row for each group.

<center><img src="imgs/image_0.png" width=40%></center>

- Instead of aggregating during the apply step, we could instead perform a **filtration**, in which we keep only the groups that satisfy some condition.

- Or a **transformation**, in which we perform operations to every value within each group.

### Grouping, then filtering

- To keep only the groups that satisfy a particular condition, use the `filter` method on a `DataFrameGroupBy`/`SeriesGroupBy` object.<br><small>The `filter` method takes in a function, which itself takes in a DataFrame/Series and return a single Boolean. The result is a new DataFrame/Series with only the groups for which the filter function returned `True`.</small>

- For example, suppose we want only the `'species'` whose average `'bill_length_mm'` is above 39.

In [ ]:
(
    penguins
    .groupby('species')
    .filter(lambda df: df['bill_length_mm'].mean() > 39)
)

- No more `'Adelie'`s!

<div class="alert alert-success">
<h3>Activity</h3>

Create a new DataFrame with only the rows in `penguins` for popular `'species'` – that is, `'species'` with at least 100 penguins.

In [ ]:
(
    penguins
    .groupby('species')
    .filter(lambda df: df.shape[0] >= 100)
)

In [ ]:
# Note that to just find the 'species' with at least 100 penguins,
# we didn't need to group:
penguins['species'].value_counts()

<div class="alert alert-danger" markdown="1">

#### Reference Slide

### Example: Z-Scoring

- Suppose we want to convert the `'body_mass_g'` column to to z-scores (i.e. standard units):

$$z(x_i) = \frac{x_i - \text{mean of } x}{\text{SD of } x}$$

In [ ]:
def z_score(x):
    return (x - x.mean()) / x.std(ddof=0)

In [ ]:
z_score(penguins['body_mass_g'])

<div class="alert alert-danger" markdown="1">

#### Reference Slide

### Grouping, then transforming

- Now, what if we wanted the z-score within each group?

- To do so, we can use the `transform` method on a `DataFrameGroupBy` object. The `transform` method takes in a function, which itself takes in a Series and returns a new Series.

- A transformation produces a DataFrame or Series of the same size – it is **not** an aggregation!

In [ ]:
z_mass = (penguins
          .groupby('species')
          ['body_mass_g']
          .transform(z_score))
z_mass

In [ ]:
penguins.assign(z_mass=z_mass)

In [ ]:
display_df(penguins.assign(z_mass=z_mass), rows=8)

- Note that above, penguin 340 has a larger `'body_mass_g'` than penguin 0, but a lower `'z_mass'`.
    - Penguin 0 has an above average `'body_mass_g'` among `'Adelie'` penguins.
    - Penguin 340 has a below average `'body_mass_g'` among `'Gentoo'` penguins. Remember from earlier that the average `'body_mass_g'` of `'Gentoo'` penguins is much higher than for other species.

### What's next?

- Can we group on multiple columns at once?

- What does `pivot_table` do?

- How do we combine two DataFrames with information about similar individuals?

- How do we deal with missing values?

- How do we decide which type of visualization to create?